In [ ]:
#@title Installing needed dependencies!
from IPython.display import clear_output
try:
    from app_store_scraper import AppStore
    from pprint import pprint
    import pandas as pd
    import numpy as np
    import json
    print("Modules required are already installed!")
except ModuleNotFoundError:
    %pip install google-play-scraper
    clear_output()
    %pip install app-store-scraper
    clear_output()
    %pip install google-search-scraper-python
    clear_output()
    %pip install pandas
    clear_output()
    print("All modules required were sucessfully installed!")

In [ ]:
#@title App Reviews from App Store
from app_store_scraper import AppStore
from pprint import pprint
import pandas as pd
import numpy as np
import json

#@markdown * ### Input the name of the app: 
app = "sololearn" #@param {type:"string"}
applereviews = AppStore(country="us", app_name=app)
#@markdown * ### How many reviews?
count = 100000 #@param {type:"number"}
applereviews.review(how_many=count)
applereviews.reviews
applestoredf = pd.DataFrame(np.array(applereviews.reviews),columns=['review'])
applestoredf2 = applestoredf.join(pd.DataFrame(applestoredf.pop('review').tolist()))
#@markdown * ### Display data?
visualize = "Yes" #@param ["Yes", "No"]
if visualize == "Yes":
  display(applestoredf2)
else:
  print("No display")
#@markdown * ### Save to csv?
csv = "No" #@param ["Yes", "No"]
if csv == "Yes":
  applestoredf2.to_csv('applestore-review.csv') 
else:
  print("No export")

In [ ]:
#@title App Reviews from Google Store
from google_play_scraper import app, search, Sort, reviews, reviews_all
from pprint import pprint
import pandas as pd
import numpy as np
import json
#static variables:
language = 'en'
countries = 'us'
appid = []

def searches(nameofSearch):
  result = search(
    nameofSearch,
    lang="en",  # defaults to 'en'
    country="us",  # defaults to 'us'
    n_hits=3  # defaults to 30 (= Google's maximum)
)
  return result

def getAppId(r):
  for d in r:
    id = (d['appId'])
    # appid.append(id)
    return id

def insertId(g, cont, rating, choice):

  if choice == None:
    if rating == None:
      result, continuation_token = reviews(
      g,
      lang='en', # defaults to 'en'
      country='us', # defaults to 'us'
      sort=Sort.NEWEST, # defaults to Sort.NEWEST
      count=cont # defaults to 100
      # defaults to None(means all score)
    )
      return result
    else:
      result, continuation_token = reviews(
      g,
      lang='en', # defaults to 'en'
      country='us', # defaults to 'us'
      sort=Sort.NEWEST, # defaults to Sort.NEWEST
      count=cont, # defaults to 100
      filter_score_with=rating# defaults to None(means all score)
    )
      return result
 
  else:
    if rating == None:
      result = reviews_all(
      g,
      sleep_milliseconds=0, # defaults to 0
      lang='en', # defaults to 'en'
      country='us', # defaults to 'us'
      # sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
      filter_score_with=None
      )
      return result
    else: 
      result = reviews_all(
      g,
      sleep_milliseconds=0, # defaults to 0
      lang='en', # defaults to 'en'
      country='us', # defaults to 'us'
      # sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
      filter_score_with=rating # defaults to None(means all score)
      )
      return result

def googleplay(a, ct, r, c):
  searches1 = searches(a)
  appId = getAppId(searches1)
  reviews = insertId(appId, ct, r, c)
  return reviews

def googleplay1(a, ct, r, c):
  searches1 = searches(a)
  appId = getAppId(searches1)
  reviews = insertId(appId, ct, r, c)
  return reviews 

#@markdown * ### Input the name of the app: 
app = "sololearn" #@param {type:"string"}

#@markdown * ### All reviews?
choice = "No" #@param ["Yes", "No"]
if choice == "Yes":
  choice = "Yes"
  count = None
else:  
  choice = None
  #@markdown * ### How many reviews? (leave empty if you chose Yes to all reviews)
  count = 100 #@param {type:"number"}
#@markdown * ### Include specific rating?
cond = "No" #@param ["Yes", "No"]  
if cond == "Yes":
  #@markdown * ### Input the rating: (0-5) (leave empty if you chose No to specific rating)
  rating = 0 #@param {type:"integer"}
else:
  print("default rating")
  rating = None

googleplaydf = pd.DataFrame(np.array(googleplay(app, count, rating, choice)),columns=['review'])
googleplaydf2 = googleplaydf.join(pd.DataFrame(googleplaydf.pop('review').tolist()))


#@markdown * ### Display data?
visualize = "Yes" #@param ["Yes", "No"]
if visualize == "Yes":
  display(googleplaydf2)
else:
  print("No display")

#@markdown * ### Save to csv?
csv = "No" #@param ["Yes", "No"]
if csv == "Yes":
  googleplaydf2.to_csv('googleplay-review.csv') 
else:
  print("No export")


In [ ]:
import time

result = reviews_all(
      'com.sololearn',
      sleep_milliseconds=0, # defaults to 0
      lang='en', # defaults to 'en'
      country='us', # defaults to 'us'
      # sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
      # defaults to None(means all score)
      )

result1 = reviews_all(
      'com.sololearn',
      sleep_milliseconds= 0, # defaults to 0
      lang='es', # defaults to 'en'
      country='us', # defaults to 'us'
      # sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
      # defaults to None(means all score)
      )


googleplaydf = pd.DataFrame(np.array(googleplay(app, count, rating, choice)),columns=['review'])
googleplaydf2 = googleplaydf.join(pd.DataFrame(googleplaydf.pop('review').tolist()))

googleplaydf3 = pd.DataFrame(np.array(googleplay(app, count, rating, choice)),columns=['review'])
googleplaydf4 = googleplaydf3.join(pd.DataFrame(googleplaydf3.pop('review').tolist()))

df5 = googleplaydf2.append(googleplaydf4, ignore_index=True)

time.sleep(60)
display(df5)

In [ ]:
df5.to_csv('googleplay-review.csv')

In [ ]:
result = search(
    "pokemon",
    lang="en",  # defaults to 'en'
    country="us",  # defaults to 'us'
    n_hits=3  # defaults to 30 (= Google's maximum)
)
print(result)

[{'appId': 'com.nianticlabs.pokemongo', 'icon': 'https://play-lh.googleusercontent.com/MDaSgXlbRkftfjNIdJ2oHodVBVLOmVg2PevfdzJkbtlawfMA-8gMAs-kCfXXY5XyLw', 'screenshots': ['https://play-lh.googleusercontent.com/O-OR6Mh0AoNyiaYYaa3OJ_VHGfLqWW2qNzUUZxRRodD3fqs2Pm04FatavdNbz-jsMZM', 'https://play-lh.googleusercontent.com/HiLNXxmIpOl1jLBssHWcGqsQ58oUC5RvmS5tiX7L86mPHCG6wVEN-aX5OxTKAbDzh10', 'https://play-lh.googleusercontent.com/Il9B11lqrHX_Kd3QzLCA6hA6O7EsT56ItiLWMf1JkwcdmRyR3CE2KW_vR9w1izanO-JM', 'https://play-lh.googleusercontent.com/_84Pq9dJ0_iTKwx9-CkYFYoakSMKK_yfdrp1WSX-inE2XvCTr8ri2tKoISLHN_9PEvP5', 'https://play-lh.googleusercontent.com/JjgLXR7CMiQfSbtICOa86_35f7Pf8IzJn78zWQslwJn56Qm2O7BOV7xzXXE8mz3Vhg', 'https://play-lh.googleusercontent.com/iAYd9LdbFI7aajPU760XoNZ8b3woDQ58B0harYiUed2y7WQLU19fcj9I8yS-_K9BDQ', 'https://play-lh.googleusercontent.com/lpw5tz7Onf_6Sx4Q3kGX1zKXcec4EWpRyr9I4w5d3TrQMoorPWVke6veB5qmqhfQZn4', 'https://play-lh.googleusercontent.com/KgDQ-Kjb2B7_jDP-8KmQDNhAmP

In [ ]:
# !pip install pycountry
import pycountry

len(pycountry.countries)
list(pycountry.countries)

In [ ]:
result1 = reviews_all(
      'com.nianticlabs.pokemongo',
      sleep_milliseconds= 0, # defaults to 0
      # lang='es', # defaults to 'en'
      country='us', # defaults to 'us'
      # sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
      # defaults to None(means all score)
      )

googleplaydf = pd.DataFrame(np.array(result1),columns=['review'])
googleplaydf2 = googleplaydf.join(pd.DataFrame(googleplaydf.pop('review').tolist()))
display(googleplaydf2)